In [1]:
import os
import zipfile


os.makedirs('data', exist_ok=True)

# Unzip the file
with zipfile.ZipFile('/content/water segmentation yolov11.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

print("Extraction complete!")

Extraction complete!


In [2]:
!pip install ultralytics

In [3]:
from ultralytics import YOLO
import shutil

exp_name = "my_project"
model = YOLO('yolo11m-seg.pt')

model.train(
    data='data/data.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    optimizer='Adam',
    lr0=0.001,
    device='cuda:0',
    verbose=True,

    # Data augmentations
    mosaic=1.0,
    hsv_h=0.02,
    hsv_s=0.7,
    hsv_v=0.4,
    flipud=0.5,
    degrees=10,
    perspective=0.0001,

    # Overfitting prevention
    patience=30,

    # Learning rate scheduler
    cos_lr = True,

    # Output directory
    project='runs/train',
    name=exp_name,
    exist_ok=True
)

# Path to best model
best_model_path = f"runs/train/{exp_name}/weights/best.pt"

# Save best model to root directory
shutil.copy(best_model_path, "best_model.pt")
print("Best model saved as best_model.pt")

# Evaluate best model on test set with results saved
print("Evaluating best model on test set...")
best_model = YOLO("best_model.pt")  # Load the best model
metrics = best_model.val(data='data/data.yaml', split='test', device='cuda:0', save=True)

# Evaluate on validation set for overfitting check
val_metrics = best_model.val(data='data/data.yaml', split='val', device='cuda:0')

# For segmentation, use .seg metrics
test_map = metrics.seg.map50
val_map = val_metrics.seg.map50
print(f"Validation mAP50: {val_map:.3f}, Test mAP50: {test_map:.3f}")

# You can also check box metrics if needed
print(f"Test Box mAP50: {metrics.box.map50:.3f}")
print(f"Test Mask mAP50: {metrics.seg.map50:.3f}")





Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data/data.yaml, degrees=10, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.02, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=my_project, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=30, perspective=0.0001, plots=True, pose=12.0, pre